In [1]:
import main

In [2]:
es = main.SubtitlesExtractor("ttxsh.mp4")
print(es.framecount)

143699.0


In [3]:
import json
fp = open("boxes.json", "r")
boxes = json.load(fp)
fp.close()

In [4]:
subtitles = es.extract_from_box(boxes[0], 10)

AssertionError: 

In [ ]:
from Levenshtein import ratio, median

In [ ]:
class Subtitle:
    def __init__(self, text, t_start, t_end):
        self.text = text
        self.t_start = t_start
        self.t_end = t_end    

In [ ]:
def group_subtitles_by_sim(subtitles: list, min_sim):
    subtitles_groups = []
    length = len(subtitles)

    # Agrupar legendas parecidas em grupos
    x = 0
    while x < length:
        x_sub = subtitles[x]
        group = [x_sub, ]
        y = x + 1
        while y < length:
            y_sub = subtitles[y]
            if ratio(x_sub.text.replace(" ", ""), y_sub.text.replace(" ", "")) >= min_sim:
                group.append(y_sub)
            else:
                x = y
                break
            y += 1
        subtitles_groups.append(group)
        x += 1
                        
    return subtitles_groups

In [ ]:
groups = group_subtitles_by_sim(subtitles, 0.7)
for x in groups:
    for y in x:
        print(y, y.time, y.frame)
    print("\n\n")

In [ ]:
def merge_subtitles_groups_by_sim(subtitles_groups, min_sim):
    subtitles_groups = subtitles_groups.copy()
    i = 0
    while i < len(subtitles_groups) - 1:
        group = subtitles_groups[i]
        next_group = subtitles_groups[i+1]
        m = median([x.text.replace(" ", "") for x in group])
        next_m = median([x.text.replace(" ", "") for x in next_group])

        if ratio(m, next_m) >= min_sim:
            group.extend(next_group)
            subtitles_groups[i] = group
            subtitles_groups.pop(i+1)
            continue

        i += 1
    
    return subtitles_groups

In [ ]:
def remove_duplicate_subtitles(subtitles: list, min_sim=0.7):
    groups = group_subtitles_by_sim(subtitles, min_sim)
    # groups = merge_subtitles_groups_by_sim(groups, min_sim)

    result = list()

    for g in groups:
        s = max(g, key=lambda x: x.confidence)
        result.append(s)
    return result

In [ ]:
m = remove_duplicate_subtitles(subtitles)
print(m)

In [ ]:
from IPython.display import Image
from matplotlib import pyplot as plt

In [ ]:
es.video.set(cv.CAP_PROP_POS_FRAMES, 9706)
ret, frame = es.video.read()
# frame = frame[319:343, 104:355]
plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2GRAY))
plt.title("banana")
plt.show()

In [ ]:
from paddleocr import PaddleOCR

In [ ]:
ocr = PaddleOCR(lang="ch", rec=False)
res = ocr.ocr(frame)

In [ ]:
res[0][2][1]

In [ ]:
!pip install shapely